In [42]:
def btoi(x):
    return int.from_bytes(x, 'little')

In [43]:
## First, some preliminaries that will be needed.

import hashlib

def sha512(s):
    return hashlib.sha512(s).digest()

# Base field Z_p
p = 2**255 - 19

def modp_inv(x):
    return pow(x, p-2, p)

# Curve constant
d = -121665 * modp_inv(121666) % p

# Group order
q = 2**252 + 27742317777372353535851937790883648493

def sha512_modq(s):
    return int.from_bytes(sha512(s), "little") % q

## Then follows functions to perform point operations.

# Points are represented as tuples (X, Y, Z, T) of extended
# coordinates, with x = X/Z, y = Y/Z, x*y = T/Z

def point_add(P, Q):
    A, B = (P[1]-P[0]) * (Q[1]-Q[0]) % p, (P[1]+P[0]) * (Q[1]+Q[0]) % p;
    C, D = 2 * P[3] * Q[3] * d % p, 2 * P[2] * Q[2] % p;
    E, F, G, H = B-A, D-C, D+C, B+A;
    return (E*F, G*H, F*G, E*H);

# Computes Q = s * Q
def point_mul(s, P):
    Q = (0, 1, 1, 0)  # Neutral element
    while s > 0:
        if s & 1:
            Q = point_add(Q, P)
        P = point_add(P, P)
        s >>= 1
    return Q

def point_equal(P, Q):
    # x1 / z1 == x2 / z2  <==>  x1 * z2 == x2 * z1
    if (P[0] * Q[2] - Q[0] * P[2]) % p != 0:
        return False
    if (P[1] * Q[2] - Q[1] * P[2]) % p != 0:
        return False
    return True

## Now follows functions for point compression.

# Square root of -1
modp_sqrt_m1 = pow(2, (p-1) // 4, p)

# Compute corresponding x-coordinate, with low bit corresponding to
# sign, or return None on failure
def recover_x(y, sign):
    if y >= p:
        return None
    x2 = (y*y-1) * modp_inv(d*y*y+1)
    if x2 == 0:
        if sign:
            return None
        else:
            return 0

    # Compute square root of x2
    x = pow(x2, (p+3) // 8, p)
    if (x*x - x2) % p != 0:
        x = x * modp_sqrt_m1 % p
    if (x*x - x2) % p != 0:
        return None

    if (x & 1) != sign:
        x = p - x
    return x

# Base point
g_y = 4 * modp_inv(5) % p
g_x = recover_x(g_y, 0)
G = (g_x, g_y, 1, g_x * g_y % p)

def point_compress(P):
    zinv = modp_inv(P[2])
    x = P[0] * zinv % p
    y = P[1] * zinv % p
    return int.to_bytes(y | ((x & 1) << 255), 32, "little")

def point_decompress(s):
    if len(s) != 32:
        raise Exception("Invalid input length for decompression")
    y = int.from_bytes(s, "little")
    sign = y >> 255
    y &= (1 << 255) - 1

    x = recover_x(y, sign)
    if x is None:
        return None
    else:
        return (x, y, 1, x*y % p)

## These are functions for manipulating the private key.

def secret_expand(secret):
    if len(secret) != 32:
        raise Exception("Bad size of private key")
    h = sha512(secret)
    a = int.from_bytes(h[:32], "little")
    a &= (1 << 254) - 8
    a |= (1 << 254)
    return (a, h[32:])

def secret_to_public(secret):
    (a, dummy) = secret_expand(secret)
    return point_compress(point_mul(a, G))

## The signature function works as below.

def sign(secret, msg):
    a, prefix = secret_expand(secret)
    A = point_compress(point_mul(a, G))
    r = sha512_modq(prefix + msg)
    R = point_mul(r, G)
    Rs = point_compress(R)
    h = sha512_modq(Rs + A + msg)
    s = (r + h * a) % q
    return Rs + int.to_bytes(s, 32, "little")

## And finally the verification function.

def verify(public, msg, signature):
    if len(public) != 32:
        raise Exception("Bad public key length")
    if len(signature) != 64:
        Exception("Bad signature length")
    A = point_decompress(public)
    print("A:", hex(A[0]),  hex(A[1]),  hex(A[2]))
    if not A:
        return False
    Rs = signature[:32]
    print("Rs:", hex(btoi(Rs)))
    R = point_decompress(Rs)
    if not R:
        return False
    s = int.from_bytes(signature[32:], "little")
    if s >= q: return False
    toh = Rs + public + msg
    print("TOH:", hex(btoi(toh)))
    h = sha512_modq(Rs + public + msg)
    print("h:", hex(h))
    sB = point_mul(s, G)
    hA = point_mul(h, A)
    return point_equal(sB, point_add(R, hA))

In [44]:
eddsa_in = "4e7d21fb3b1897571a445833be0f9fd41cd62be3aa04040f8934e1fcbdcacd4531b2524b8348f7ab1dfafa675cc538e9a84e3fe5819e27c12ad8bbc1a36e4dff:31b2524b8348f7ab1dfafa675cc538e9a84e3fe5819e27c12ad8bbc1a36e4dff:33d7a786aded8c1bf691:28e4598c415ae9de01f03f9f3fab4e919e8bf537dd2b0cdf6e79b9e6559c9409d9151a4c40f083193937627c369488259e99da5a9f0a87497fa6696a5dd6ce0833d7a786aded8c1bf691:"

In [45]:
import sys
import binascii

def munge_string(s, pos, change):
    return (s[:pos] +
            int.to_bytes(s[pos] ^ change, 1, "little") +
            s[pos+1:])

# Read a file in the format of
# http://ed25519.cr.yp.to/python/sign.input
line = eddsa_in
fields = line.split(":")
secret = (binascii.unhexlify(fields[0]))[:32]
public = binascii.unhexlify(fields[1])
msg = binascii.unhexlify(fields[2])
signature = binascii.unhexlify(fields[3])[:64]

pubkey = secret_to_public(secret)

print("public:", hex(btoi(public)))
print("msg:", hex(btoi(msg)))
print("signature:", hex(btoi(signature)))
print()

assert public == pubkey
assert signature == sign(secret, msg)
assert verify(public, msg, signature)
if len(msg) == 0:
    bad_msg = b"x"
else:
    bad_msg = munge_string(msg, len(msg) // 3, 4)
#assert not verify(public,bad_msg,signature)
#assert not verify(public, msg, munge_string(signature,20,8))
#assert not verify(public,msg,munge_string(signature,40,16))

public: 0xff4d6ea3c1bbd82ac1279e81e53f4ea8e938c55c67fafa1dabf748834b52b231
msg: 0x91f61b8cedad86a7d733
signature: 0x8ced65d6a69a67f49870a9f5ada999e258894367c6237391983f0404c1a15d909949c55e6b9796edf0c2bdd37f58b9e914eab3f9f3ff001dee95a418c59e428

A: 0x4cd530baaf96cfce24c4baf37493d3cc26d6d2c886c8d8938da2c0c48aaa27ab 0x7f4d6ea3c1bbd82ac1279e81e53f4ea8e938c55c67fafa1dabf748834b52b231 0x1
Rs: 0x9949c55e6b9796edf0c2bdd37f58b9e914eab3f9f3ff001dee95a418c59e428
TOH: 0x91f61b8cedad86a7d733ff4d6ea3c1bbd82ac1279e81e53f4ea8e938c55c67fafa1dabf748834b52b23109949c55e6b9796edf0c2bdd37f58b9e914eab3f9f3ff001dee95a418c59e428
h: 0xc41c35527fc3ecea13195c06574ced8b3e32a3b5ea082e240e3aa36aadcd50a


In [67]:
Ginv = ((0 - G[0]) % p, G[1], G[2], (G[0] * (0 - G[1]) % p) % p)
#Ginv = (G[0], (0 - G[1]) % p, G[2], (G[0] * (0 - G[1]) % p) % p)
print(G)
print(Ginv)
O = point_add(G, Ginv)
Zinv = modp_inv(O[2]) % p
print(hex((O[0] * Zinv) % p ), hex((O[1] * Zinv) % p))

(15112221349535400772501151409588531511454012693041857206046113283949847762202, 46316835694926478169428394003475163141307993866256225615783033603165251855960, 1, 46827403850823179245072216630277197565144205554125654976674165829533817101731)
(42783823269122696939284341094755422415180979639778424813682678720006717057747, 46316835694926478169428394003475163141307993866256225615783033603165251855960, 1, 11068640767834918466713275874066756361490786778694627043054626174422747718218)
0x0 0x1


In [63]:
(Ginv[1] + G[1]) % p

0

In [57]:
a = 0x22
ma = (0 - a) % p


In [58]:
(a + ma) % p

0